In [17]:
import cv2
import time
import numpy as np
from skimage.measure import compare_ssim

# в обох варіантах використаємо підхід, що полягає в видаленні кожного 2 кадру. також всі важливі кадри будемо зберігати в папках decoded, original та differ


In [18]:
def encode(out, cap, path):
    i = 0
    while (True):
        ret, img = cap.read()
        if (ret == 0): break
        if (i % 2 == 0 ):
            out.write(img)
        else:
            cv2.imwrite(path + f"{int(i/2)}.png", img)
        i += 1

    cap.release()
    out.release()

In [19]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('encoded.avi',fourcc, 20.0, (320,240))

cap = cv2.VideoCapture('car.avi')



In [20]:
encode(out, cap, "original/original")

# зсуваємо кожен піксель на flow порахований методом farneback

In [24]:
def apply_flow1(img, flow):
    tmp = img
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            for k in range(img.shape[2]):
                try:
                    tmp[int(i+flow[i,j,0]),int(j+flow[i,j,1]), k] = img[i,j,k]
                except: pass
                '''
                Exception as e:
                    print(i,j,k)
                    print(img[i,j,k])
                    print(flow[i,j,0], flow[i,j,1])
                    '''
    return tmp

# декодер бере 2 збережених кадра та відновлює, той що був між ними

In [25]:
def decode1(out, cap, path):
    t1 = time.time()
    ret, prev = cap.read()
    prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    out.write(prev)
    i = 0
    while (True):
        ret, img = cap.read()
        if (ret == 0): break
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        prevgray = gray

        pred = apply_flow1(prev, flow)

        cv2.imshow('pred', pred)
        prev = img
        out.write(pred)
        cv2.imwrite(path + f"{i}.png", pred)
        i += 1
        out.write(img)

        ch = 0xFF & cv2.waitKey(5)
        if ch == 27:
            break
    t2 = time.time()

    cap.release()
    out.release()     
    cv2.destroyAllWindows()     
    return (t2 - t1)/ i

In [23]:
#import numpy as np
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('decoded1.avi',fourcc, 20.0, (320,240))

cap = cv2.VideoCapture('encoded.avi')
time1 = decode1(out, cap, "decoded1/decoded")

In [15]:
print(time1)

1.0161097010179443


# обраховуємо візуальну різницю між оригіналом та відновленим кадром

In [26]:
def difference(path1, path2, path3):
    q = 0
    scores = []
    while True:
        if (q == 784): break
        before = cv2.imread(path1 + f"{q}.png", cv2.IMREAD_COLOR)
        after = cv2.imread(path2 + f"{q}.png", cv2.IMREAD_COLOR)

        # Convert images to grayscale
        before_gray = cv2.cvtColor(before, cv2.COLOR_BGR2GRAY)
        after_gray = cv2.cvtColor(after, cv2.COLOR_BGR2GRAY)

        # Compute SSIM between two images
        (score, diff) = compare_ssim(before_gray, after_gray, full=True)
        #print("Image similarity", score)
        scores.append(score)

        # The diff image contains the actual image differences between the two images
        # and is represented as a floating point data type in the range [0,1] 
        # so we must convert the array to 8-bit unsigned integers in the range
        # [0,255] before we can use it with OpenCV
        diff = (diff * 255).astype("uint8")

    # Threshold the difference image, followed by finding contours to
    # obtain the regions of the two input images that differ
        thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = contours[0] if len(contours) == 2 else contours[1]

        filled_after = after.copy()

        for c in contours:
            area = cv2.contourArea(c)
            if area > 40:
                x,y,w,h = cv2.boundingRect(c)
                cv2.rectangle(after, (x, y), (x + w, y + h), (36,255,12), 2)
                cv2.drawContours(filled_after, [c], 0, (0,150,0), -1)

        for i in range(before.shape[0]):
            for j in range(before.shape[1]):
                for k in range(before.shape[2]):
                    filled_after[i,j,k] = (int(filled_after[i,j,k]) + int(after[i,j,k])) / 2


        cv2.imwrite(path3 + f"{q}.png", filled_after)
        q += 1
    scores = np.array(scores)
    return np.mean(scores)

In [27]:
score1 = difference("original/original", "decoded1/decoded", "differ1/differ")

<ipython-input-26-ecf515911434>:14: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  (score, diff) = compare_ssim(before_gray, after_gray, full=True)


In [28]:
print(score1)

0.6789816860917643


# другий метод на основі Lucas-Kanade

In [29]:
def apply_flow2(img, p0, p1):
	tmp = img
	q0 = np.array(p0)
	q1 = np.array(p1)
	e = 2*(q1 - q0)
	for i in range(e.shape[0]):

		try:

			for q in range(int(q0[i,0,0]) - 5, int(q0[i,0,0]) +5):

				for p in range(int(q0[i,0,1]) - 5, int(q0[i,0,1]) +5):

					for k in range(3):

						tmp[q + int(e[i,0,0]), p + int(e[i,0,1]),k] = img[q, p,k]
		except: pass
	#except: print('##')
	return tmp

In [37]:
def decode2(out, cap, path):
    t1 = time.time()
    feature_params = dict( maxCorners = 500,
                                                 qualityLevel = 0.2,
                                                 minDistance = 10,
                                                 blockSize = 20)
    # Parameters for lucas kanade optical flow
    lk_params = dict( winSize  = (15,15),
                                        maxLevel = 2,
                                        criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    # Create some random colors
    color = np.random.randint(0,255,(100,3))
    # Take first frame and find corners in it
    ret, prev = cap.read()
    old_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
    # Create a mask image for drawing purposes
    #mask = np.zeros_like(old_frame)
    i = 0
    while(1):
            ret,frame = cap.read()
            if ret == 0: break
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            #try:
            # calculate optical flow
            p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
            # Select good points

            pred = apply_flow2(prev, p0, p1)
            prev =  frame

            out.write(pred)
            out.write(prev)
            cv2.imwrite(path + f"{i}.png", pred)

            #cv.imshow('frame',img)
            k = cv2.waitKey(30) & 0xff
            if k == 27:
                    break
            # Now update the previous frame and previous points
            old_gray = frame_gray.copy()
            p0 = p1
            '''
            except: 
                out.write(frame)
                cv.imwrite(f'decoded/decoded{i}.png', frame)
                p0 = cv.goodFeaturesToTrack(frame_gray, mask = None, **feature_params)
                old_gray = frame_gray.copy()
                print('###')
                '''
            i += 1
    t2 = time.time()

    cap.release()
    out.release()     
    cv2.destroyAllWindows()     
    return (t2 - t1)/ i

In [38]:
#import numpy as np
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('decoded2.avi',fourcc, 20.0, (320,240))

cap = cv2.VideoCapture('encoded.avi')
time2 = decode2(out, cap, "decoded2/decoded")

In [39]:
print(time2)

0.02160183233874185


In [40]:
score2 = difference("original/original", "decoded2/decoded", "differ2/differ")
print(score2)

<ipython-input-26-ecf515911434>:14: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  (score, diff) = compare_ssim(before_gray, after_gray, full=True)


0.9717302168699871


#  ми бачимо, що farneback працює довше за рахонук зміщення всіх пікселів зображення. але мінусом є те, що не завжди зміщення є правильно нормлваним і пікселі перескакують надто далеко. Lucas-Kanade навпаки не завжди дектектить ключові точки і врезультвті ми бачимо кадр майже ідентичний попередньому. В обох методах є потенціал для покращення, що звичайно ж приведе до більшої складності алгоритму